# Recitation 14!?
Traits and Generics

## Traits

Traits are an important mechanism for code reuse in scala. They allow us to define functionality that can be 
exported across multiple objects in the overall hierarchy. A trait is almost like an abstract class or an interface. It can define its own members and methods.

For this exercise, we have defined two traits: `NumberOfLegs` that helps us define how many legs a given animal has and `WarmBlooded` that applies to warm blooded animals.  We also have an abstract class `Animal` as a superclass for all animals.

In [9]:
abstract class Animal

trait NumberOfLegs { //not extends because may or may not be an animal
    val nLegs: Int
    def getNumberOfLegs: Int = nLegs
}

trait WarmBlooded extends Animal {
    val bodyTempMaintained: Double
    def getBodyTemp: Double = bodyTempMaintained 
}

defined class Animal
defined trait NumberOfLegs
defined trait WarmBlooded

We will define a class `Human` which will extend the appropriate classes and traits defined above.
Our class will also take in a parameter called `Name` of type `String` and implement a `getName` method without any parameters.

Which traits should we use for our human class? (Discuss this first then move onto the coding of the class)

In [12]:
class Human(Name: String) extends Animal with WarmBlooded with NumberOfLegs {
    def getName: String = {
        Name
    }
    override val nLegs= 2
    override val bodyTempMaintained= 98.0
}

defined class Human

In [13]:
//BEGIN TEST
val t1 = new Human("Jane Smith")
assert(t1.getNumberOfLegs == 2, "Your human does not have two legs")
assert(t1.bodyTempMaintained == 98.0, "Your human does not maintain a body temp of 98")
assert(t1.getBodyTemp == 98.0, "Your human's getBodyTemp Function is not working")
assert(t1.getName == "Jane Smith", "Your human's name is not setting correctly")
//END TEST

t1: Human = ammonite.$sess.cmd11$Helper$Human@65fcec24

Now we will define a class named `Table`.

Which of the traits and classes should we extend? (discuss this first then advance to the coding) (if someone complains about the number of legs on a table, adjust the exercise so that the table class takes in the number of legs, and update the test)

In [14]:
// Write the Table class here, using the discussed traits
class Table extends NumberOfLegs {
    override val nLegs=4
}

defined class Table

In [15]:
//BEGIN TEST
val tbl = new Table()
assert(tbl.getNumberOfLegs == 4, "A Table must have four legs")
//END TEST

tbl: Table = ammonite.$sess.cmd13$Helper$Table@a5b3092

We may want to do certain things based on a more precise type than a trait gives us, like only the humans in a list of instances of NumberOfLegs.
While not as easy as pattern matching with case classes, this is possible by using the `isInstanceOf[T]` and the `asInstanceOf[T]` functions.

The expression `obj.isInstanceOf[T]` returns true if `obj` is an instance of `T` and false otherwise.
Similarly, `obj.asInstanceOf[T]` returns the same instance `obj`, but with the given type `T` (if it is an instance of `T`).

To try out these functions, lets write a function to get the names of all the humans in a list of instances of the NumberOfLegs trait.

In [18]:
def getNames(leggedThings : List[NumberOfLegs]): List[String] = {
    leggedThings match {
        case h :: t => {
            if (h.isInstanceOf[Human]) {
                val myHuman= h.asInstanceOf[Human]
                val humName= myHuman.getName
                val listofNames= humName::getNames(t) 
                //returning list with humans name with the rest of the names in the list
                listofNames
            }
            else {
                getNames(t) //not a human so move on
            }
        }
        case Nil => Nil
    }
}

defined function getNames

In [19]:
// TEST CASE
val leggedThings : List[NumberOfLegs] = List(new Human("Charles"), new Human("Sukanya"), new Table(), new Table(), new Human("Abhishek"))
assert(getNames(leggedThings) == List("Charles", "Sukanya", "Abhishek"), "Names don't match")

leggedThings: List[NumberOfLegs] = List(
  ammonite.$sess.cmd11$Helper$Human@31e9abb1,
  ammonite.$sess.cmd11$Helper$Human@289fa35d,
  ammonite.$sess.cmd13$Helper$Table@799aefcd,
  ammonite.$sess.cmd13$Helper$Table@434b86a7,
  ammonite.$sess.cmd11$Helper$Human@9da8229
)

## Generics

Generic classes and functions are classes that take a type as a parameter.
We put these type parameters in square brackets (for example `List[String]` or `asInstanceOf[Human]` where we are passing `String` or `Human`, respectively)

One common use case for generics is for collections like maps, trees, and lists.
We will further exam generics using the example of lists.

If we wanted to define a list of integers, we could do something like this:

In [20]:
sealed trait IntList
case class IntNil() extends IntList
case class IntCons(head: Int, tail: IntList) extends IntList

defined trait IntList
defined class IntNil
defined class IntCons

But what if we needed a list of strings instead?
We could write the following:

In [21]:
sealed trait StringList
case class StringNil() extends StringList
case class StringCons(head: String, tail: StringList) extends StringList

defined trait StringList
defined class StringNil
defined class StringCons

What about booleans? Or your custom class you wrote, such as `Human` above?
We don't want to redefine these traits and the functions that work with them everytime we need a list for a different type.
We could define a list holding objects of type `Any` (the supertype of all types), but this has problems when we try access the objects we put in the list.

Instead we use generics.
We do this by adding type parameters to our code.

Coming back to our list example, we could write it using generics as follows:

In [22]:
sealed trait GenericList[T]

case class GenericNil[T]() extends GenericList[T]
case class GenericCons[T](h: T, t: GenericList[T]) extends GenericList[T]

defined trait GenericList
defined class GenericNil
defined class GenericCons

The type parameter `[T]` can be replaced with any class we want, either explicitly or by the type scala infers:

In [23]:
val explicit = GenericCons[String]("hello", GenericCons[String]("world", GenericNil[String]()))
val inferred = GenericCons("hello", GenericCons("world", GenericNil()))

explicit: GenericCons[String] = GenericCons(
  h = "hello",
  t = GenericCons(h = "world", t = GenericNil())
)
inferred: GenericCons[String] = GenericCons(
  h = "hello",
  t = GenericCons(h = "world", t = GenericNil())
)

In [24]:
val l = GenericCons[Integer](1, GenericNil())

l: GenericCons[Integer] = GenericCons(h = 1, t = GenericNil())

As an exercise with our `GenericList`, let's define the map function for `GenericList`s

In [25]:
def map[A,B](list: GenericList[A], function: (A) => B): GenericList[B] =  {
    list match {
        case GenericNil() => GenericNil[B]()
        case GenericCons(head,tail) => {
            val genB= function(head)
            val mapTail=map(tail,function)
            GenericCons[B](genB, mapTail)
        
        }
    }
}

defined function map

### Type Bounds (If time permits)
We can also apply restrictions on the the types used for our type parameters

We can use `<:` to ensure that the type parameter is a subtype of the given type, or `:>` to ensure that the type parameter is a supertype of the given type.

In [ ]:
sealed trait LegList[T <: NumberOfLegs]

case class LegNil[T <: NumberOfLegs]() extends LegList[T]
case class LegCons[T <: NumberOfLegs](h: T, t: LegList[T]) extends LegList[T]

In [ ]:
def countLegs[T <: NumberOfLegs](leggedThings : LegList[T]) : Int = leggedThings match {
    case LegNil() => 0
    case LegCons(h, t) => h.getNumberOfLegs + countLegs(t)
}

In [ ]:
def countLegs[T <: NumberOfLegs](leggedThings : List[T]): Int = {
    leggedThings.foldLeft[Int](0)((accum, leggedThing) => accum + leggedThing.getNumberOfLegs)
}

Note that the ever useful `foldLeft` function also has a type parameter, which lets `foldLeft` return any type we want.  Of course, the compiler usually infers the type parameter's value, so we don't need include the type.